In [1]:
import sys
sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')

In [2]:
#import h5py

from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))

gm.draw_grid()

In [3]:
cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))

In [4]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [5]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [6]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()
grid_domain = GridDomain(domain_size=(3.,2.), integer_size=(3,2))
grid_domain.make_pointwise_index(ma)

In [7]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)
deep = MA.assemble()

[(0, 2), (1, 16), (2, 17), (3, 18), (4, 0), (5, 24), (6, 25), (7, 19), (8, 1), (9, 26), (10, 27), (11, 20), (12, 3), (13, 22), (14, 23), (15, 21)] [(0, 6), (1, 4), (2, 5), (3, 2), (4, 7), (5, 12), (6, 13), (7, 0), (8, 8), (9, 14), (10, 15), (11, 1), (12, 9), (13, 10), (14, 11), (15, 3)]
[(0, 2), (1, 16), (2, 17), (3, 18), (4, 0), (5, 24), (6, 25), (7, 19), (8, 1), (9, 26), (10, 27), (11, 20), (12, 3), (13, 22), (14, 23), (15, 21)] [(0, 6), (1, 4), (2, 5), (3, 2), (4, 7), (5, 12), (6, 13), (7, 0), (8, 8), (9, 14), (10, 15), (11, 1), (12, 9), (13, 10), (14, 11), (15, 3)]
{0, 1, 2, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 25, 26, 27} {32, 33, 34, 35, 36, 40, 41, 42, 43, 28, 29, 31} [[37, 38, 39, 30]] [[9, 10, 11, 3, 22, 23, 21]]
 2

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [9]:
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

0 X derivative: 1.4326159342159528e-08
Y derivative: -1.1560971222013272e-08
XX derivative: -0.06944445258134024
YY derivative: -0.07100851460213832
XY_derivative: 1.9691274687205773e-09

1 X derivative: -3.5501590542330597e-08
Y derivative: 2.454490749537186e-09
XX derivative: -0.06944444365495747
YY derivative: -0.14856425286631894
XY_derivative: -3.132565484786909e-09

2 X derivative: 0.1666666661164693
Y derivative: -8.648394639321211e-10
XX derivative: -0.01388888740748527
YY derivative: -0.010253045015680005
XY_derivative: -1.0757421776280696e-10

3 X derivative: -1.7669437302192392e-09
Y derivative: -1.87534846679549e-09
XX derivative: -0.020833321974344756
YY derivative: 0.0611978813047196
XY_derivative: -2.232580578742649e-09

4 X derivative: 0.4166666641518922
Y derivative: -0.0027072280989555945
XX derivative: -0.034722217950292525
YY derivative: -0.03710536215468046
XY_derivative: 0.0027072342058133813

5 X derivative: 0.4166666641696824
Y derivative: 0.0014758840311768773


In [ ]:
partial_diff_test(matrix=deep[0][-1], point=grid_domain.normal_index[2], pointnum=2, pointdict=grid_domain.pointwise_index)

In [ ]:
dist = distributed_eye_easy(ma.get_cell_list_of_ddofs(cell=cell3), axis2shape=MA.assembly_interface.get_ddof_count())

In [ ]:
pipka = dist.T * csr_matrix(MA._get_local_gg_matrix(distribution='lobatto', order=3)) * dist

In [ ]:
mm = np.array(MA.assembled_mass.sum(axis=1)).flatten()

In [ ]:
mm_sp = csr_matrix(np.diag(1/mm))

In [ ]:
op =  mm_sp * MA.assembled 

In [ ]:
grid_domain.normal_index

In [ ]:
MA.assembly_interface.allocator.get_cell_list_of_ddofs(cell2)

In [ ]:
plt.imshow(deep[0][-1].toarray(), interpolation='none')

In [ ]:
grid_domain.normal_index